# 데이터 특성 처리 통합 실습

이 노트북은 범주형 데이터 처리에 대한 다양한 방법들을 학습합니다.

## 학습 목표
1. **범주형 데이터의 기본 처리**: `pd.get_dummies()` 사용법
2. **숫자형 범주 데이터 처리**: `OneHotEncoder` 활용
3. **통합 전처리**: `ColumnTransformer`로 컬럼별 전처리 적용

## 범주형 데이터 처리의 필요성

범주형 자료는 다음과 같은 경우에 특별한 처리가 필요합니다:
- 문자열로 된 범주형 데이터
- 숫자 형태의 범주형 데이터 (예: 1.대, 2.중, 3.소)

### 문제점
직업분류가 1~16의 숫자로 코딩된 경우:
- 1보다 16이 더 큰 값으로 인식되어 모델이 잘못된 중요도를 학습할 수 있음
- 해결방법: **원핫인코딩(One-Hot Encoding)** 적용


In [1]:
# 필요한 라이브러리 임포트
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import os

# mglearn 라이브러리가 설치되어 있지 않은 경우 아래 명령어로 설치
# !pip install mglearn
try:
    import mglearn 
    print("mglearn 라이브러리 로드 성공")
except ImportError:
    print("mglearn 라이브러리가 설치되어 있지 않습니다. '!pip install mglearn'으로 설치해주세요.")
    
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer


mglearn 라이브러리 로드 성공


## 1. pandas get_dummies()를 사용한 범주형 데이터 처리

### Adult 데이터셋 로드 및 전처리

Adult 데이터셋을 사용하여 범주형 데이터 처리를 실습해보겠습니다.


In [2]:
# Adult 데이터셋 로드
try:
    data = pd.read_csv(os.path.join(mglearn.datasets.DATA_PATH, "adult.data"), 
                       header=None, index_col=False, 
                       names=['age', 'workclass', 'fnlwgt', 'education', 'education-num',
                              'marital-status', 'occupation', 'relationship', 'race', 'gender',
                              'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
                              'income'])
    print("데이터셋 로드 성공!")
    print("원본 데이터 형태:")
    print(data.head())
    
except Exception as e:
    print(f"데이터셋 로드 실패: {e}")
    print("mglearn 라이브러리의 데이터를 사용할 수 없습니다. 대신 샘플 데이터를 생성합니다.")
    
    # 샘플 데이터 생성
    np.random.seed(42)
    n_samples = 1000
    
    data = pd.DataFrame({
        'age': np.random.randint(18, 80, n_samples),
        'workclass': np.random.choice(['Private', 'Self-emp-not-inc', 'Self-emp-inc', 'Federal-gov', 'Local-gov', 'State-gov'], n_samples),
        'education': np.random.choice(['Bachelors', 'HS-grad', 'Masters', 'Some-college', '11th', 'Doctorate'], n_samples),
        'gender': np.random.choice(['Male', 'Female'], n_samples),
        'hours-per-week': np.random.randint(20, 80, n_samples),
        'occupation': np.random.choice(['Tech-support', 'Craft-repair', 'Exec-managerial', 'Sales', 'Other-service'], n_samples),
        'income': np.random.choice(['<=50K', '>50K'], n_samples)
    })
    print("샘플 데이터 생성 완료!")
    print("샘플 데이터 형태:")
    print(data.head())


데이터셋 로드 성공!
원본 데이터 형태:
   age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital-status          occupation    relationship    race   gender  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital-gain  capital-loss  hours-per-week  native-country  income  
0          2174             0              40

In [3]:
# 분석에 사용할 컬럼 선택 (마지막 필드가 타겟)
data = data[['age', 'workclass', 'education', 'gender', 'hours-per-week', 'occupation', 'income']]
print("선택된 컬럼들:")
print(data.head())
print(f"\n데이터 형태: {data.shape}")
print(f"\n데이터 타입:")
print(data.dtypes)


선택된 컬럼들:
   age          workclass   education   gender  hours-per-week  \
0   39          State-gov   Bachelors     Male              40   
1   50   Self-emp-not-inc   Bachelors     Male              13   
2   38            Private     HS-grad     Male              40   
3   53            Private        11th     Male              40   
4   28            Private   Bachelors   Female              40   

           occupation  income  
0        Adm-clerical   <=50K  
1     Exec-managerial   <=50K  
2   Handlers-cleaners   <=50K  
3   Handlers-cleaners   <=50K  
4      Prof-specialty   <=50K  

데이터 형태: (32561, 7)

데이터 타입:
age                int64
workclass         object
education         object
gender            object
hours-per-week     int64
occupation        object
income            object
dtype: object


In [4]:
# get_dummies 함수를 사용한 원핫인코딩
print("=== get_dummies 적용 전 ===")
print(f"원본 데이터 컬럼 수: {len(data.columns)}")
print(f"컬럼명: {list(data.columns)}")

# 원핫인코딩 적용
data_encoded = pd.get_dummies(data)
print(f"\n=== get_dummies 적용 후 ===")
print(f"인코딩 후 컬럼 수: {len(data_encoded.columns)}")
print(f"처음 5개 컬럼: {list(data_encoded.columns[:5])}")
print(f"마지막 5개 컬럼: {list(data_encoded.columns[-5:])}")

print(f"\n인코딩된 데이터 미리보기:")
print(data_encoded.head())


=== get_dummies 적용 전 ===
원본 데이터 컬럼 수: 7
컬럼명: ['age', 'workclass', 'education', 'gender', 'hours-per-week', 'occupation', 'income']

=== get_dummies 적용 후 ===
인코딩 후 컬럼 수: 46
처음 5개 컬럼: ['age', 'hours-per-week', 'workclass_ ?', 'workclass_ Federal-gov', 'workclass_ Local-gov']
마지막 5개 컬럼: ['occupation_ Sales', 'occupation_ Tech-support', 'occupation_ Transport-moving', 'income_ <=50K', 'income_ >50K']

인코딩된 데이터 미리보기:
   age  hours-per-week  workclass_ ?  workclass_ Federal-gov  \
0   39              40         False                   False   
1   50              13         False                   False   
2   38              40         False                   False   
3   53              40         False                   False   
4   28              40         False                   False   

   workclass_ Local-gov  workclass_ Never-worked  workclass_ Private  \
0                 False                    False               False   
1                 False                    False       

In [5]:
# 타겟 변수와 특성 변수 분리
# 주의: get_dummies가 타겟까지 원핫인코딩한 상태
print("=== X, y 분리 ===")

# income 관련 컬럼들 찾기
income_columns = [col for col in data_encoded.columns if col.startswith('income_')]
feature_columns = [col for col in data_encoded.columns if not col.startswith('income_')]

print(f"타겟 관련 컬럼: {income_columns}")
print(f"특성 컬럼 수: {len(feature_columns)}")

# X, y 분리
X = data_encoded[feature_columns]
# income_>50K 또는 첫 번째 income 컬럼을 타겟으로 사용
if 'income_ >50K' in data_encoded.columns:
    y = data_encoded['income_ >50K']
elif 'income_>50K' in data_encoded.columns:
    y = data_encoded['income_>50K']
else:
    y = data_encoded[income_columns[0]]  # 첫 번째 income 컬럼 사용

print(f"\nX 형태: {X.shape}")
print(f"y 형태: {y.shape}")
print(f"\nX 첫 5행:")
print(X.head())
print(f"\ny 첫 5개 값:")
print(y.head())


=== X, y 분리 ===
타겟 관련 컬럼: ['income_ <=50K', 'income_ >50K']
특성 컬럼 수: 44

X 형태: (32561, 44)
y 형태: (32561,)

X 첫 5행:
   age  hours-per-week  workclass_ ?  workclass_ Federal-gov  \
0   39              40         False                   False   
1   50              13         False                   False   
2   38              40         False                   False   
3   53              40         False                   False   
4   28              40         False                   False   

   workclass_ Local-gov  workclass_ Never-worked  workclass_ Private  \
0                 False                    False               False   
1                 False                    False               False   
2                 False                    False                True   
3                 False                    False                True   
4                 False                    False                True   

   workclass_ Self-emp-inc  workclass_ Self-emp-not-inc  workclass_

In [7]:
# 로지스틱 회귀 모델 학습 (수렴 문제 해결)
print("=== 로지스틱 회귀 모델 학습 (개선된 버전) ===")

# 1단계: 데이터 스케일링 (수렴 문제 해결을 위함)
print("\n📊 데이터 스케일링 적용...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(f"스케일링 전 X 범위: age({X['age'].min()}-{X['age'].max()}), hours({X['hours-per-week'].min()}-{X['hours-per-week'].max()})")
print(f"스케일링 후 X 범위: 평균=0, 표준편차=1로 정규화")

# 2단계: 개선된 로지스틱 회귀 모델
print("\n🤖 로지스틱 회귀 모델 학습...")
model = LogisticRegression(
    max_iter=5000,        # 반복 횟수 증가
    solver='liblinear',   # 이진 분류에 효과적인 솔버
    C=1.0,               # 정규화 강도
    random_state=42      # 재현성을 위한 시드
)

model.fit(X_scaled, y)
print("✅ 모델 학습 완료! (수렴 경고 없음)")

# 3단계: 모델 성능 평가
print("\n📈 모델 성능 평가:")
accuracy = model.score(X_scaled, y)
print(f"모델 정확도: {accuracy:.4f} ({accuracy*100:.2f}%)")

# 4단계: 예측 결과 확인
y_pred = model.predict(X_scaled)
y_pred_proba = model.predict_proba(X_scaled)[:, 1]  # 확률 예측

print(f"\n🎯 예측 결과 샘플:")
print(f"실제 값:     {y[:10].values}")
print(f"예측 값:     {y_pred[:10]}")
print(f"예측 확률:   {[f'{p:.3f}' for p in y_pred_proba[:10]]}")

# 5단계: 정확도 분석
from sklearn.metrics import classification_report, confusion_matrix
print(f"\n📊 상세 성능 분석:")
print(classification_report(y, y_pred, target_names=['<=50K', '>50K']))

# 혼동 행렬
cm = confusion_matrix(y, y_pred)
print(f"\n혼동 행렬:")
print(f"           예측")
print(f"실제    <=50K  >50K")
print(f"<=50K   {cm[0,0]:5d} {cm[0,1]:5d}")
print(f">50K    {cm[1,0]:5d} {cm[1,1]:5d}")


=== 로지스틱 회귀 모델 학습 (개선된 버전) ===

📊 데이터 스케일링 적용...
스케일링 전 X 범위: age(17-90), hours(1-99)
스케일링 후 X 범위: 평균=0, 표준편차=1로 정규화

🤖 로지스틱 회귀 모델 학습...
✅ 모델 학습 완료! (수렴 경고 없음)

📈 모델 성능 평가:
모델 정확도: 0.8126 (81.26%)

🎯 예측 결과 샘플:
실제 값:     [False False False False False False False  True  True  True]
예측 값:     [False False False False False False False False False  True]
예측 확률:   ['0.294', '0.405', '0.080', '0.068', '0.161', '0.357', '0.005', '0.426', '0.298', '0.612']

📊 상세 성능 분석:
              precision    recall  f1-score   support

       <=50K       0.84      0.93      0.88     24720
        >50K       0.67      0.43      0.53      7841

    accuracy                           0.81     32561
   macro avg       0.76      0.68      0.70     32561
weighted avg       0.80      0.81      0.80     32561


혼동 행렬:
           예측
실제    <=50K  >50K
<=50K   23067  1653
>50K     4450  3391


### 🔧 수렴 문제 해결 방법

위의 개선된 코드에서 다음과 같은 방법으로 수렴 문제를 해결했습니다:

#### 1️⃣ **데이터 스케일링 (StandardScaler)**
- **문제**: `age`(18-90)와 `hours-per-week`(1-99) 등 숫자형 특성들이 서로 다른 스케일을 가짐
- **해결**: 모든 특성을 평균=0, 표준편차=1로 정규화
- **효과**: 로지스틱 회귀의 수렴 속도 향상

#### 2️⃣ **모델 파라미터 조정**
- **max_iter**: 1000 → 5000 (반복 횟수 증가)
- **solver**: 'lbfgs' → 'liblinear' (이진 분류에 최적화된 솔버)
- **random_state**: 42 (재현 가능한 결과)

#### 3️⃣ **상세한 성능 분석**
- **분류 보고서**: 정밀도, 재현율, F1-score 제공
- **혼동 행렬**: 예측 오류 패턴 분석
- **예측 확률**: 모델의 확신도 확인

#### ⚡ **결과**
- ✅ 수렴 경고 해결
- 📈 더 안정적인 모델 성능
- 🎯 상세한 예측 분석 가능


## 2. OneHotEncoder를 사용한 숫자형 범주 데이터 처리

### 문제 상황
특성이 원래는 범주형인데 숫자 형태로 입력되는 경우, `get_dummies()`는 이를 범주로 인식하지 못합니다.

### 해결 방법
1. 숫자형 범주 데이터를 문자열로 변환
2. `OneHotEncoder` 클래스 사용


In [8]:
# 숫자형 범주 데이터 예제
demo_df = pd.DataFrame({
    '숫자특성': [0, 1, 2, 1],  # 범주형이지만 숫자로 표현
    '범주형특성': ['양말', '여우', '양말', '상자']    
})

print("=== 원본 데모 데이터 ===")
print(demo_df)
print(f"\n데이터 타입:")
print(demo_df.dtypes)


=== 원본 데모 데이터 ===
   숫자특성 범주형특성
0     0    양말
1     1    여우
2     2    양말
3     1    상자

데이터 타입:
숫자특성      int64
범주형특성    object
dtype: object


In [9]:
# get_dummies의 문제점: 숫자형 범주 데이터를 범주로 인식하지 못함
df1 = pd.get_dummies(demo_df)
print("=== get_dummies 적용 결과 ===")
print(df1)
print("\n문제점: '숫자특성'은 그냥 숫자로 처리되고, '범주형특성'만 원핫인코딩됨")

# 해결방법 1: 숫자형 범주 데이터를 문자열로 변환
print("\n=== 해결방법 1: 문자열 변환 후 get_dummies ===")
demo_df_str = demo_df.copy()
demo_df_str['숫자특성'] = demo_df_str['숫자특성'].astype(str)
df2 = pd.get_dummies(demo_df_str)
print(df2)


=== get_dummies 적용 결과 ===
   숫자특성  범주형특성_상자  범주형특성_양말  범주형특성_여우
0     0     False      True     False
1     1     False     False      True
2     2     False      True     False
3     1      True     False     False

문제점: '숫자특성'은 그냥 숫자로 처리되고, '범주형특성'만 원핫인코딩됨

=== 해결방법 1: 문자열 변환 후 get_dummies ===
   숫자특성_0  숫자특성_1  숫자특성_2  범주형특성_상자  범주형특성_양말  범주형특성_여우
0    True   False   False     False      True     False
1   False    True   False     False     False      True
2   False   False    True     False      True     False
3   False    True   False      True     False     False


In [10]:
# 해결방법 2: OneHotEncoder 클래스 사용
print("=== 해결방법 2: OneHotEncoder 사용 ===")

# sparse_output=False: 희소행렬 대신 numpy 배열 반환
ohe = OneHotEncoder(sparse_output=False)

# fit_transform: fit과 transform을 한번에 수행
encoded_array = ohe.fit_transform(demo_df)

print("인코딩된 특성명:")
print(ohe.get_feature_names_out())

print("\n인코딩된 결과 (numpy 배열):")
print(encoded_array)

# DataFrame으로 변환하여 보기 좋게 출력
encoded_df = pd.DataFrame(encoded_array, columns=ohe.get_feature_names_out())
print("\n인코딩된 결과 (DataFrame):")
print(encoded_df)


=== 해결방법 2: OneHotEncoder 사용 ===
인코딩된 특성명:
['숫자특성_0' '숫자특성_1' '숫자특성_2' '범주형특성_상자' '범주형특성_양말' '범주형특성_여우']

인코딩된 결과 (numpy 배열):
[[1. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 1.]
 [0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0.]]

인코딩된 결과 (DataFrame):
   숫자특성_0  숫자특성_1  숫자특성_2  범주형특성_상자  범주형특성_양말  범주형특성_여우
0     1.0     0.0     0.0       0.0       1.0       0.0
1     0.0     1.0     0.0       0.0       0.0       1.0
2     0.0     0.0     1.0       0.0       1.0       0.0
3     0.0     1.0     0.0       1.0       0.0       0.0


## 3. ColumnTransformer를 사용한 통합 전처리

`ColumnTransformer`는 컬럼별로 다른 전처리 작업을 적용할 수 있는 강력한 도구입니다.

### 사용 시나리오
- 숫자형 특성: 표준화(StandardScaler)
- 범주형 특성: 원핫인코딩(OneHotEncoder)
- 각 컬럼에 맞는 전처리를 한번에 적용


In [11]:
# ColumnTransformer 실습을 위한 데이터 준비
# 앞에서 사용한 data를 다시 사용 (income 컬럼 포함)
print("=== ColumnTransformer 실습용 데이터 ===")

# 원본 데이터에서 타겟을 제외한 특성들만 선택
ct_data = data.drop('income', axis=1).copy()
print("ColumnTransformer 적용 전 데이터:")
print(ct_data.head())
print(f"\n데이터 형태: {ct_data.shape}")
print(f"컬럼 정보:")
print(ct_data.dtypes)


=== ColumnTransformer 실습용 데이터 ===
ColumnTransformer 적용 전 데이터:
   age          workclass   education   gender  hours-per-week  \
0   39          State-gov   Bachelors     Male              40   
1   50   Self-emp-not-inc   Bachelors     Male              13   
2   38            Private     HS-grad     Male              40   
3   53            Private        11th     Male              40   
4   28            Private   Bachelors   Female              40   

           occupation  
0        Adm-clerical  
1     Exec-managerial  
2   Handlers-cleaners  
3   Handlers-cleaners  
4      Prof-specialty  

데이터 형태: (32561, 6)
컬럼 정보:
age                int64
workclass         object
education         object
gender            object
hours-per-week     int64
occupation        object
dtype: object


In [12]:
# ColumnTransformer 정의
print("=== ColumnTransformer 정의 ===")

# 숫자형 컬럼과 범주형 컬럼 구분
numeric_columns = ['age', 'hours-per-week']
categorical_columns = ['workclass', 'education', 'gender', 'occupation']

print(f"숫자형 컬럼: {numeric_columns}")
print(f"범주형 컬럼: {categorical_columns}")

# ColumnTransformer 생성
# 각 전처리 단계를 (이름, 변환기, 컬럼들) 튜플로 정의
ct = ColumnTransformer([
    ("scaling", StandardScaler(), numeric_columns),
    ("onehot", OneHotEncoder(sparse_output=False), categorical_columns)
])

print("\nColumnTransformer 설정 완료!")
print("- 숫자형 컬럼: StandardScaler 적용")
print("- 범주형 컬럼: OneHotEncoder 적용")


=== ColumnTransformer 정의 ===
숫자형 컬럼: ['age', 'hours-per-week']
범주형 컬럼: ['workclass', 'education', 'gender', 'occupation']

ColumnTransformer 설정 완료!
- 숫자형 컬럼: StandardScaler 적용
- 범주형 컬럼: OneHotEncoder 적용


In [13]:
# ColumnTransformer 적용
print("=== ColumnTransformer 적용 ===")

# fit: 각 변환기에 필요한 통계 정보 학습
ct.fit(ct_data)
print("fit 완료: 각 변환기의 파라미터 학습됨")

# transform: 실제 변환 수행
transformed_data = ct.transform(ct_data)
print(f"\n변환 완료!")
print(f"원본 데이터 형태: {ct_data.shape}")
print(f"변환된 데이터 형태: {transformed_data.shape}")

# 변환된 데이터의 첫 5행 확인
print(f"\n변환된 데이터 (첫 5행):")
print(transformed_data[:5])

# 특성명 확인
feature_names = ct.get_feature_names_out()
print(f"\n변환된 특성명 (총 {len(feature_names)}개):")
print(f"처음 10개: {feature_names[:10]}")
print(f"마지막 10개: {feature_names[-10:]}")


=== ColumnTransformer 적용 ===
fit 완료: 각 변환기의 파라미터 학습됨

변환 완료!
원본 데이터 형태: (32561, 6)
변환된 데이터 형태: (32561, 44)

변환된 데이터 (첫 5행):
[[ 0.03067056 -0.03542945  0.          0.          0.          0.
   0.          0.          0.          1.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          1.          0.          0.          0.
   0.          0.          0.          0.          1.          0.
   1.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.83710898 -2.22215312  0.          0.          0.          0.
   0.          0.          1.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          1.          0.          0.          0.
   0.          0.          0.          0.          1.          0.
   0.          0.          0.          1.          0.    

## 4. 정리 및 비교

### 세 가지 방법 비교

| 방법 | 장점 | 단점 | 사용 시점 |
|------|------|------|-----------|
| **pd.get_dummies()** | - 간단하고 직관적<br>- pandas와 잘 통합됨 | - 숫자형 범주 데이터 인식 못함<br>- 컬럼별 다른 처리 어려움 | - 모든 컬럼이 문자열 범주형<br>- 간단한 전처리 |
| **OneHotEncoder** | - 숫자형 범주 데이터 처리 가능<br>- scikit-learn 파이프라인과 호환 | - 별도 클래스 사용 필요<br>- pandas DataFrame과 분리됨 | - 숫자형 범주 데이터 포함<br>- ML 파이프라인 구축 |
| **ColumnTransformer** | - 컬럼별 다른 전처리 가능<br>- 복잡한 전처리 파이프라인 구축<br>- 재사용성 높음 | - 설정이 복잡<br>- 초보자에게 어려움 | - 혼합된 데이터 타입<br>- 복잡한 전처리 필요 |

### 권장 사용법
1. **간단한 범주형 데이터**: `pd.get_dummies()`
2. **숫자형 범주 데이터**: `OneHotEncoder`
3. **복잡한 혼합 데이터**: `ColumnTransformer`


In [14]:
# 실습 요약
print("=" * 50)
print("         데이터 특성 처리 실습 완료!")
print("=" * 50)

print("\n✅ 완료된 실습:")
print("1. pd.get_dummies()로 Adult 데이터셋 처리")
print("2. OneHotEncoder로 숫자형 범주 데이터 처리")  
print("3. ColumnTransformer로 통합 전처리")

print("\n📊 처리된 데이터 크기:")
print(f"- 원본 Adult 데이터: {data.shape}")
print(f"- get_dummies 후: {data_encoded.shape}")
print(f"- ColumnTransformer 후: {transformed_data.shape}")

print("\n🎯 핵심 학습 내용:")
print("- 범주형 데이터의 올바른 처리 방법")
print("- 숫자형 범주 데이터의 주의점")
print("- 상황에 맞는 전처리 도구 선택")

print("\n다음 단계: 이 전처리된 데이터로 머신러닝 모델을 학습해보세요!")


         데이터 특성 처리 실습 완료!

✅ 완료된 실습:
1. pd.get_dummies()로 Adult 데이터셋 처리
2. OneHotEncoder로 숫자형 범주 데이터 처리
3. ColumnTransformer로 통합 전처리

📊 처리된 데이터 크기:
- 원본 Adult 데이터: (32561, 7)
- get_dummies 후: (32561, 46)
- ColumnTransformer 후: (32561, 44)

🎯 핵심 학습 내용:
- 범주형 데이터의 올바른 처리 방법
- 숫자형 범주 데이터의 주의점
- 상황에 맞는 전처리 도구 선택

다음 단계: 이 전처리된 데이터로 머신러닝 모델을 학습해보세요!
